In [1]:
import gym
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import torch

In [2]:
env = gym.make('FrozenLake-v1')

In [3]:
_ = env.reset()

In [4]:
n_obs = env.observation_space.n
n_act = env.action_space.n
gamma = 1
max_iters = 10000
TOL  = 1e-10

In [5]:
def policy_evaluation(policy):
    v_s = np.zeros(n_obs)
    for i in range(1000):
        v_copy = np.copy(v_s)
        for s in range(n_obs):
            a = policy[s]
            rewards = 0
            for trans_prob, s_next, r, done in env.P[s][a]:
                rewards += trans_prob * (r + gamma * v_copy[s_next])
            v_s[s] = rewards

        if abs(v_s - v_copy).max() < TOL:
            break
    return v_s

In [6]:
def policy_improvement(v_s):
    policy = np.zeros(n_obs)
    for s in range(n_obs):
        q_sa = []
        for a in range(n_act):
            rewards = 0
            for trans_prob, s_next, r, done in env.P[s][a]:
                rewards += trans_prob * (r + gamma * v_s[s_next])
            q_sa.append(rewards)
        policy[s] = np.argmax(q_sa)
    return policy

In [7]:
policy = np.zeros(n_obs)
for i in range(max_iters):
    v_s = policy_evaluation(policy)
    update_policy = policy_improvement(v_s)
    
    if abs(update_policy - policy).max() < TOL:
        break
    
    policy = np.copy(update_policy)

In [8]:
reward_list = []
for i in range(100):
    done = False
    s = env.reset()
    rewards = []
    while not done:
        s, r, done, _ = env.step(int(policy[s]))
        rewards.append(r)
    reward_list.append(rewards)